## VOSA input format

VOSA requires following format for uploading the photometric information files

| object  | RA  | DEC | dis | Av | filter | flux | error | pntopts | objopts |
| ---     | --- | --- | --- | ---| ---    | ---  | ---   | ---     | ---     |
| ---     | --- | --- | --- | ---| ---    | ---  | ---   | ---     | ---     |

Identify the filters in the files you are uploading (create the "filter_list" accordingly)

Create a file with "name, ra, dec, magnitudes and magnitude errors".
This "photomety_file" be converted to VOSA format
    Note: This code is for a cluster, hence distance and extinction is kept constant
    
The VOSA_input.txt file has magnitudes in the "flux" column. So while uploading to VOSA, keep "file type" as "magnitudes"

## Read input file

In [44]:
import pandas as pd
import numpy as np
def reprint(message):
    print ('\r %s' %(message), end='')

df_photometry = pd.read_csv('data/example_photomety_file.csv', engine='python')
df_photometry.set_index('name', inplace=True)
distance      = '831.76+-11'  # [pc]
Av           = '0.1736+-0.017'    # 3.1 * E(B-V) [mag]

print((df_photometry.columns))
df_photometry.head()

Index(['ra', 'dec', 'B', 'e_B', 'V', 'e_V', 'I', 'e_I', 'U', 'e_U', 'R', 'e_R',
       'F148W', 'e_F148W', 'F154W', 'e_F154W', 'F169M', 'e_F169M'],
      dtype='object')


,ra,dec,B,e_B,V,e_V,I,e_I,U,e_U,R,e_R,F148W,e_F148W,F154W,e_F154W,F169M,e_F169M
name,,,,,,,,,,,,,,,,,,
WOCS2002,132.8492,11.83040,12.300,NaN,11.540,NaN,11.060,NaN,NaN,NaN,NaN,NaN,18.66000,0.058,18.69300,0.065,18.58200,0.058
WOCS1007,132.8931,11.85297,11.070,NaN,10.960,NaN,11.072,NaN,NaN,NaN,NaN,NaN,17.03341,0.018,16.87171,0.016,16.63688,0.019
Y1168,132.8331,11.81147,18.409,NaN,18.632,NaN,NaN,NaN,18.393,NaN,NaN,NaN,17.02925,0.039,17.06582,0.033,17.17563,0.033


## Rename columns
- Use filter names as given in http://svo2.cab.inta-csic.es/theory/fps/
- *Required* columns: filter_name, e_filter_name, distance, Av
- If there are unnecessary columns, no need to change them



In [47]:
df_photometry.columns = ['ra', 'dec', 
                         'KPNO/Mosaic.B', 'e_KPNO/Mosaic.B', 'KPNO/Mosaic.V', 'e_KPNO/Mosaic.V', 
                         'KPNO/Mosaic.I', 'e_KPNO/Mosaic.I', 'KPNO/Mosaic.U','e_KPNO/Mosaic.U', 
                         'KPNO/Mosaic.R', 'e_KPNO/Mosaic.R', 
                         'Astrosat/UVIT.F148W', 'e_Astrosat/UVIT.F148W', 'Astrosat/UVIT.F154W', 'e_Astrosat/UVIT.F154W', 
                         'Astrosat/UVIT.F169M', 'e_Astrosat/UVIT.F169M']
filter_list = ['KPNO/Mosaic.B', 'KPNO/Mosaic.V','KPNO/Mosaic.I', 'KPNO/Mosaic.U','KPNO/Mosaic.R',
               'Astrosat/UVIT.F148W','Astrosat/UVIT.F154W', 'Astrosat/UVIT.F169M']
df_photometry['distance'] = distance
df_photometry['Av'] = Av

df_photometry.head()

,ra,dec,KPNO/Mosaic.B,e_KPNO/Mosaic.B,KPNO/Mosaic.V,e_KPNO/Mosaic.V,KPNO/Mosaic.I,e_KPNO/Mosaic.I,KPNO/Mosaic.U,e_KPNO/Mosaic.U,KPNO/Mosaic.R,e_KPNO/Mosaic.R,Astrosat/UVIT.F148W,e_Astrosat/UVIT.F148W,Astrosat/UVIT.F154W,e_Astrosat/UVIT.F154W,Astrosat/UVIT.F169M,e_Astrosat/UVIT.F169M,distance,Av
name,,,,,,,,,,,,,,,,,,,,
WOCS2002,132.8492,11.83040,12.300,NaN,11.540,NaN,11.060,NaN,NaN,NaN,NaN,NaN,18.66000,0.058,18.69300,0.065,18.58200,0.058,831.76+-11,0.1736+-0.017
WOCS1007,132.8931,11.85297,11.070,NaN,10.960,NaN,11.072,NaN,NaN,NaN,NaN,NaN,17.03341,0.018,16.87171,0.016,16.63688,0.019,831.76+-11,0.1736+-0.017
Y1168,132.8331,11.81147,18.409,NaN,18.632,NaN,NaN,NaN,18.393,NaN,NaN,NaN,17.02925,0.039,17.06582,0.033,17.17563,0.033,831.76+-11,0.1736+-0.017


In [57]:
def create_VOSA_input_file(df_photometry, filter_list, output_name=None):
    # combining data from all stars to make the VOSA upload file 
    df_VOSA_input = pd.DataFrame(columns = ['object', 'ra', 'dec','distance','Av','filter','flux','error','pntopts','objopts']) 
    counter=0
    for name in df_photometry.index:
        reprint(name)
        for filter_name in filter_list:
            df_VOSA_input.loc[counter, 'object']   = name
            df_VOSA_input.loc[counter, 'ra']       = df_photometry['ra'][name]
            df_VOSA_input.loc[counter, 'dec']      = df_photometry['dec'][name]
            df_VOSA_input.loc[counter, 'distance'] = df_photometry['distance'][name]
            df_VOSA_input.loc[counter, 'Av']       = df_photometry['Av'][name]
            df_VOSA_input.loc[counter, 'filter']   = filter_name
            df_VOSA_input.loc[counter, 'flux']     = df_photometry[filter_name][name]
            df_VOSA_input.loc[counter, 'error']    = df_photometry['e_'+filter_name][name]
            df_VOSA_input.loc[counter, 'pntopts']  = '---'
            df_VOSA_input.loc[counter, 'objopts']  = '---'

            # Rewriting the row if flux is not available for this star-filter combination
            if ~np.isnan(df_photometry[filter_name][name]):
                counter=counter+1
    df_VOSA_input.fillna('---', inplace=True)
    if output_name !=None:
        df_VOSA_input.to_csv(output_name, header=None, index=None, sep=' ')
    return df_VOSA_input

create_VOSA_input_file(df_photometry, filter_list, output_name='data/example_VOSA_input_file.txt')

 Y1168007

,object,ra,dec,distance,Av,filter,flux,error,pntopts,objopts
0,WOCS2002,132.8492,11.83040,831.76+-11,0.1736+-0.017,KPNO/Mosaic.B,12.30000,---,---,---
1,WOCS2002,132.8492,11.83040,831.76+-11,0.1736+-0.017,KPNO/Mosaic.V,11.54000,---,---,---
2,WOCS2002,132.8492,11.83040,831.76+-11,0.1736+-0.017,KPNO/Mosaic.I,11.06000,---,---,---
3,WOCS2002,132.8492,11.83040,831.76+-11,0.1736+-0.017,Astrosat/UVIT.F148W,18.66000,0.058,---,---
4,WOCS2002,132.8492,11.83040,831.76+-11,0.1736+-0.017,Astrosat/UVIT.F154W,18.69300,0.065,---,---
5,WOCS2002,132.8492,11.83040,831.76+-11,0.1736+-0.017,Astrosat/UVIT.F169M,18.58200,0.058,---,---
6,WOCS1007,132.8931,11.85297,831.76+-11,0.1736+-0.017,KPNO/Mosaic.B,11.07000,---,---,---
7,WOCS1007,132.8931,11.85297,831.76+-11,0.1736+-0.017,KPNO/Mosaic.V,10.96000,---,---,---
8,WOCS1007,132.8931,11.85297,831.76+-11,0.1736+-0.017,KPNO/Mosaic.I,11.07200,---,---,---
9,WOCS1007,132.8931,11.85297,831.76+-11,0.1736+-0.017,Astrosat/UVIT.F148W,17.03341,0.018,---,---



Now upload the file at http://svo2.cab.inta-csic.es/theory/vosa/index.php?action=myfiles&otype=star&seeall= 

Make sure to change the File type: To magnitude (or Flux (jy) or Flux (erg) accordingly)

Keep SED Type: Flux vs Lambda 

Select the file and search through VO for all possible detections Look at the SEDs, 

Possibly remove some telescopes (e.g. SDSS photometry is problematic for bright M67 stars)


## Create flux corrected files from VOSA

In [58]:
def create_extinction_corrected_files_from_VOSA(name, output_name, verbose=True):
    # Reading parameters derived by VOSA
    file_name    = DIR_OBS + name +'/bestfitp/'+ name +'.bfit.phot.dat'
    flux         = pd.read_csv(file_name, engine='python', comment='#', delim_whitespace= True, skipinitialspace=True, header=None)
    flux.columns = ['FilterID','wavelength','Obs.Flux','Obs.Error','flux','error','model_flux_A','Fitted','Excess','FitExc','UpLim']

    # Removing filters noted as "upper limit"
    if verbose: print('WARNING: %s removed due to upper limit'%(flux[flux['UpLim']=='1']['FilterID'].values))
    flux         = flux[flux['UpLim'] == '---']
    flux         = flux.drop(columns=['Obs.Flux','Obs.Error','Fitted','Excess','FitExc','UpLim','model_flux_A'])
    flux.to_csv(output_name, index=False)
    return flux

DIR_OBS = 'data/vosa_results_53985/objects/'
for idx, name in enumerate(df_photometry.index[:]):    
    print(name)
    flux = create_extinction_corrected_files_from_VOSA(name,output_name='data/extinction_corrected_flux_files/'+name+'.csv')
    
flux

WOCS2002
WOCS1007
Y1168


,FilterID,wavelength,flux,error
0,Astrosat/UVIT.F148W,1481.000000,1.144225e-14,4.110094e-16
1,Astrosat/UVIT.F154W,1541.000000,1.023050e-14,3.109472e-16
2,Astrosat/UVIT.F169M,1608.000000,8.621919e-15,2.620557e-16
3,GALEX/GALEX.NUV,2303.366368,3.299835e-15,8.410700e-17
4,KPNO/Mosaic.U,3630.255003,2.199450e-16,3.190901e-17
5,KPNO/Mosaic.B,4357.276538,3.352139e-16,4.863190e-17
6,GAIA/GAIA3.Gbp,5035.750275,2.225325e-16,2.934945e-17
7,KPNO/Mosaic.V,5366.240786,1.571710e-16,2.280194e-17
8,GAIA/GAIA3.G,5822.388714,9.211700e-17,3.274936e-19
9,GAIA/GAIA3.Grp,7619.959993,3.964208e-17,2.414555e-18
